In [14]:
#!pip install -q transformers faiss-cpu sentence-transformers
#!pip install -U bitsandbytes

!pip install -q faiss-cpu sentence-transformers transformers bitsandbytes accelerate streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 90.5 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained("thrishala/mental_health_chatbot", quantization_config=bnb_config, device_map="auto")

# Load the model and tokenizer
model_name = "thrishala/mental_health_chatbot"
tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForCausalLM.from_pretrained(model_name)

# Ensure model is on GPU if available
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model.to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/712 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

In [ ]:
print("✅ Model loaded successfully:")
print(model)

print("\n✅ Tokenizer loaded successfully:")
print(tokenizer)


✅ Model loaded successfully:
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear8bitLt(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,

In [ ]:
input_text = "[INST] I feel unhappy [/INST]"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

with torch.no_grad():
    output = model.generate(
    **inputs,
    max_length=60,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    temperature=0.7,
    top_p=0.9
)

print("🗨️ Bot reply:", tokenizer.decode(output[0], skip_special_tokens=True))


🗨️ Bot reply: [INST] I feel unhappy [/INST] I'm sorry to hear that you're feeling unhappy. Can you please share more details about the situation or the factors that are causing this unhappiness? This will help me better understand and provide you with more relevant guidance.


In [ ]:
%%writefile app.py
import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Model & Tokenizer from Hugging Face
model_name = "thrishala/mental_health_chatbot"

@st.cache_resource
def load_model_and_tokenizer():
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    return tokenizer, model

tokenizer, model = load_model_and_tokenizer()

st.title("🧠 Mental Health Counselor Chatbot (llama2-finetuned)")
st.caption("🤖 A supportive AI companion trained on counseling and empathy data.")

# Initialize chat history
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

# Display previous messages
for role, message in st.session_state.chat_history:
    with st.chat_message(role):
        st.markdown(message)

# Chat input
if prompt := st.chat_input("You:"):
    # Show user message
    with st.chat_message("user"):
        st.markdown(prompt)

    # Build full prompt from history
    full_prompt = ""
    for role, message in st.session_state.chat_history:
        if role == "user":
            full_prompt += f"User: {message} <|sep|> "
        elif role == "bot":
            full_prompt += f"Bot: {message} <|sep|> "
    full_prompt += f"User: {prompt} <|sep|> Bot:"

    # Tokenize and generate
    input_ids = tokenizer.encode(full_prompt, return_tensors="pt")
    prompt_length = input_ids.shape[1]

    output_ids = model.generate(
        input_ids,
        max_new_tokens=200,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        #early_stopping=True
    )


    # Decode and extract response
    decoded = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    reply = decoded.split("<|sep|> Bot:")[-1].strip()

    #decoded = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    #reply = decoded.split("User:")[0].strip()


    # Show and store bot reply
    with st.chat_message("bot"):
        st.markdown(reply)

    st.session_state.chat_history.append(("user", prompt))
    st.session_state.chat_history.append(("bot", reply))


Overwriting app.py


In [ ]:
!npm install localtunnel
!streamlit run app.py --server.address=localhost &>/content/logs.txt &
!npx localtunnel --port 8501 & curl https://loca.lt/mytunnelpassword

⠙⠹⠸⠼⠴
up to date, audited 23 packages in 738ms
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠦⠙your url is: https://bumpy-areas-fly.loca.lt
104.199.168.57/content/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:15131 (check your firewall settings)
    at Socket.<anonymous> (/content/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (node:events:524:28)
    at emitErrorNT (node:internal/streams/destroy:169:8)
    at emitErrorCloseNT (node:internal/streams/destroy:128:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:82:21)

Node.js v20.19.0
⠙

In [ ]:
#Other resources
'''
https://github.com/Cody-Lange/MentalHealthAssistant
https://huggingface.co/thrishala/mental_health_chatbot
https://huggingface.co/safehavens/safehavens_chatbot
https://huggingface.co/zementalist/llama-3-8B-chat-psychotherapist
https://huggingface.co/tanusrich/Mental_Health_Chatbot
https://huggingface.co/datasets/Amod/mental_health_counseling_conversations
https://huggingface.co/datasets/Amod/mental_health_counseling_conversations
https://huggingface.co/datasets/ShenLab/MentalChat16K
'''